# Anatomist notebook extension demo

This notebook demonstrates the notebook variant of Anatomist. It runs inside `Jupyter notebook`. Paths in the demo assume that the notebook is run inside a [casa-distro container](https://github.com/brainvisa/casa-distro), and a recent build in python3 (jupyter extensions have been added only in python3 in this container).

In [12]:
import os
casa = os.environ['CASA_HOST_DIR']
doc = os.path.join(casa, 'build/share/doc/pyanatomist-5.1/sphinx/pyanatomist_notebook.html')
print('Doc is available at the following address:')
print('file://' + doc)
print('You can copy it and paste it in your web browser.')

Doc is available at the following address:
file:///home/dr144257/casa_distro/brainvisa-master-ubuntu-18.04/build/share/doc/pyanatomist-5.1/sphinx/pyanatomist_notebook.html
You can copy it and paste it in your web browser.


The API is exactly the same as the "regular" Anatomist:

In [13]:
import anatomist.notebook as ana
a = ana.Anatomist()

In [14]:
mri = a.loadObject('/casa/host/build/share/brainvisa-share-5.1/anatomical_templates/MNI152_T1_1mm.nii.gz')
w1 = a.createWindow('Axial')
w2 = a.createWindow('Coronal')
mri.addInWindows([w1, w2])
w1.moveLinkedCursor([70., 120., 75.])

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

AnatomistInteractiveWidget(height=336, layout=Layout(height='auto', width='auto'), width=424)

No handles with labels found to put in legend.


As you will note, views here are interactive and are displaying standard Anatomist views. 3D hardware is used when possible as in the headless variant of Anatomist. Displaying is a blit slower (thus less smooth) however because rendering is performed on server side, then compressed in JPEG, and transfered to the web browser.

"expensive" renderings such as Volume Rendering can be done without more overhead:

In [15]:
vol_render = a.fusionObjects([mri], method='VolumeRenderingFusionMethod')
cut_vr = a.fusionObjects([vol_render], method='FusionClipMethod')
w3 = a.createWindow('3D')
w3.addObjects(cut_vr)
w3.setControl('CutControl')

No handles with labels found to put in legend.


AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

The view above is also interactive (use middle button to rotate, shift+middle to orient the cut plane, ctrl+middle to shift it)

Other kind of windows can be displayed also, but are not refreshed as often as needed. Only user input triggers updates (but moving the mouse focus in/out is enough):

In [16]:
browser = a.createWindow('Browser')
hist = a.createWindow('Matplotlib-histogram')
mri.addInWindows([browser, hist])

AnatomistInteractiveWidget(height=400, layout=Layout(height='auto', width='auto'), width=400)

AnatomistInteractiveWidget(height=520, layout=Layout(height='auto', width='auto'), width=640)

No handles with labels found to put in legend.


Of course, meshes can also be displayed (as well as all objects supported by Anatomist):

In [17]:
mesh_l = a.loadObject('/casa/host/build/share/brainvisa-share-5.1/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lwhite_spam.gii')
mesh_r = a.loadObject('/casa/host/build/share/brainvisa-share-5.1/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rwhite_spam.gii')
w4 = a.createWindow('3D')
w4.addObjects([mesh_l, mesh_r])

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

No handles with labels found to put in legend.
